In [1]:
# from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup

In [7]:
import numpy as np
import pandas as pd

In [3]:
from transliterate import translit

In [4]:
from time import sleep

In [5]:
import re

### Avito

In [6]:
# Что первично можно выцепить на сайте:
# Город, тип дома, кол-во комнат

In [20]:
np.random.normal(1,2,1)[0]

-0.9242648604381867

In [18]:
base_url = 'https://www.avito.ru/'

In [48]:
def get_similar_flats(city, street, n_rooms, house_type):
    base_url = 'https://www.avito.ru/'
    city =  re.findall(r'[a-z- ]+', translit(city.lower(), reversed=True))[0]
    street = street.lower()
    rooms = {
        1: '1-komnatnye',
        2: '2-komnatnye',
        3: '3-komnatnye',
        4: '4-komnatnye'
    }   
    house_types = {
        'Кирпичный': 'kirpichnyy_dom',
        'Панельный': 'panelnyy_dom',
        'Блочный': 'blochnyy_dom',
        'Монолитный': 'monolitnyy_dom',
        'Деревянный': 'derevyannyy_dom'
    }   
    search = base_url + city + '/kvartiry/prodam/' + rooms[n_rooms] + '/' + house_types[house_type] + '?q=' + '+'.join(street.split()) + '&s=104'
    print(search)
    html_doc =  requests.get(search)
    # Проверка на корректность запроса
    if html_doc.status_code == 429:
        print('Сервер забанил за подозрительную активность! Разбанит примерно через час')
    else:
        print('Кажется, всё ок. Но вот на всякий случай код: {}'.format(html_doc.status_code))
    # Первая страница поиска Авито
    soup = BeautifulSoup(html_doc.text, "html.parser")
    # Нахожу все подходящие (не рекомендации) квартиры на этой странице
    flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
    flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив

    # Получаю сайт с каждой квартирой отдельно
    flats_sites = []
    for flat in flats_act:
        flats_sites.append('https://www.avito.ru' + flat.find(attrs={"class":"iva-item-sliderLink-bJ9Pv"})['href'])
    res = []
    for flat in flats_sites[:5]:
        flat_i = BeautifulSoup(requests.get(flat).text, "html.parser")
        sleep(6.451 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
        flat_one_info = {
                        'floor':None,
                        'all_floors':None,
                        'rooms':None,
                        'all_square':None,
                        'live_square':None,
                        'kitch_square':None,
                        'tall':None,
                        'repair':None,
                        'bathroom':None,
                        'balcony':None,
                        'window':None,
                        'dop_info':None,
                        'house_type':None,
                        'year_built':None,
                        'how_pas_lift':None,
                        'how_grz_lift':None,
                        'in_house':None,
                        'backyard':None,
                        'park':None,
                        'flat_lat':None,
                        'flat_lon':None,
                        'address':None,
                        'price':None,
                        'link':None
                        }

        for text in flat_i.find_all('li', class_ = 'item-params-list-item'):
            label = text.text[:text.text.find(':')].split()[0]
            value = text.text[text.text.find(':') + 1:].split()
            if label == 'Этаж':
                flat_one_info['floor'] = [value[0]]
                flat_one_info['all_floors'] = [value[2]]
            elif label == 'Количество':
                flat_one_info['rooms'] = [value[0]]
            elif label == 'Общая':
                flat_one_info['all_square'] = [value[0]]
            elif label == 'Площадь':
                flat_one_info['live_square'] = [value[0]]
            elif label == 'Жилая':
                flat_one_info['kitchen_square'] = [value[0]]
            elif label == 'Высота':
                flat_one_info['tall'] = [value[0]]   
            elif label == 'Ремонт':
                flat_one_info['repair'] = [value[0]]
            elif label == 'Санузел':
                flat_one_info['bathroom'] = [value[0]]
            elif label == 'Балкон':
                flat_one_info['balcony'] = [value[0]]
            elif label == 'Вид':
                flat_one_info['window'] = [' '.join(value)]
            elif label == 'Дополнительно':
                flat_one_info['dop_info'] = [' '.join(value)]
            elif label == 'Тип':
                flat_one_info['house_type'] = [value[0]]
            elif label == 'Год':
                flat_one_info['year_built'] = [value[0]]
            elif label == 'Пассажирский':
                flat_one_info['how_pas_lift'] = [value[0]]
            elif label == 'Грузовой':
                flat_one_info['how_grz_lift'] = [value[0]]
            elif label == 'В':
                flat_one_info['in_house'] = [' '.join(value)]
            elif label == 'Двор':
                flat_one_info['backyard'] = [' '.join(value)]
            elif label == 'Парковка':
                flat_one_info['park'] = [' '.join(value) ]
            flat_one_info['price'] = [int(flat_i.find('span', class_ = 'js-item-price')['content'])]
        flat_one_info['address'] = [flat_i.find('span', class_ = 'item-address__string').text.strip()]
        flat_one_info['flat_lat'] = [flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lat']]
        flat_one_info['flat_lon'] = [flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lon']]
        flat_one_info['link'] = [flat]
        df = pd.DataFrame(flat_one_info)
        res.append(df)
        print(' ---------- done! ----------')
        sleep(4.322 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
    return pd.concat(res)

In [45]:
res = get_similar_flats('Хабаровск', 'Гамарника', 1, 'Кирпичный')

https://www.avito.ru/habarovsk/kvartiry/prodam/1-komnatnye/kirpichnyy_dom?q=гамарника&s=104
Кажется, всё ок. Но вот на всякий случай код: 200
 ---------- done! ----------
 ---------- done! ----------
 ---------- done! ----------
 ---------- done! ----------
 ---------- done! ----------


In [49]:
pd.concat(res)

,floor,all_floors,rooms,all_square,live_square,kitch_square,tall,repair,bathroom,balcony,...,how_grz_lift,in_house,backyard,park,flat_lat,flat_lon,address,price,link,kitchen_square
0,7,16,1,38,None,None,None,евро,совмещенный,балкон,...,1,None,"закрытая территория, детская площадка, спортив...",за шлагбаумом во дворе,48.469536,135.076569,"Хабаровский край, Хабаровск, ул. Гамарника, 8",6600000,https://www.avito.ru/habarovsk/kvartiry/1-k._k...,NaN
0,13,22,1,55.5,17,None,None,требует,совмещенный,лоджия,...,2,консьерж,"закрытая территория, детская площадка",None,48.474764,135.084727,"Хабаровский край, Хабаровск, ул. Гамарника, 64",7000000,https://www.avito.ru/habarovsk/kvartiry/1-k._k...,23
0,10,10,1,36.4,9,None,3,None,None,лоджия,...,None,None,"закрытая территория, детская площадка",за шлагбаумом во дворе,48.467566,135.078942,"Хабаровск, переулок Дзержинского, 13",6200000,https://www.avito.ru/habarovsk/kvartiry/1-k._k...,NaN
0,2,5,1,30.9,None,None,None,None,совмещенный,нет,...,None,None,None,None,48.478199,135.088158,"Хабаровский край, Хабаровск, ул. Гамарника, 49",4300000,https://www.avito.ru/habarovsk/kvartiry/1-k._k...,NaN
0,4,5,1,30.4,6.1,None,None,евро,совмещенный,балкон,...,None,None,None,открытая во дворе,48.470099,135.074818,"Хабаровский край, Хабаровск, ул. Гамарника, 15А",5670000,https://www.avito.ru/habarovsk/kvartiry/1-k._k...,13


#### Получение координат из адреса по Гугл-картам

In [44]:
url = 'https://www.google.com/maps/search/' + 'Пензенская обл, г Кузнецк, ул 60-летия ВЛКСМ, д 8'

In [47]:
html_doc =  requests.get(url)

# Первая страница поиска Авито
soup = BeautifulSoup(html_doc.text, "html.parser")

In [88]:
source_lat = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[0])
source_lon = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[1])